In [ ]:
import json
import os
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

# Paths
annotations_path = './mnt/V1/annotations/annotations_preprocessed.json'
image_dir = './mnt/V1/images'

# Diccionario para mapear posiciones a índices
position_to_index = {
    '5050_guard': 0,
    'back1': 1,
    'back2': 2,
    'closed_guard1': 3,
    'closed_guard2': 4,
    'half_guard1': 5,
    'half_guard2': 6,
    'mount1': 7,
    'mount2': 8,
    'open_guard1': 9,
    'open_guard2': 10,
    'side_control1': 11,
    'side_control2': 12,
    'standing': 13,
    'takedown1': 14,
    'takedown2': 15,
    'turtle1': 16,
    'turtle2': 17
}


In [2]:
def load_annotations(annotations_path):
    with open(annotations_path, 'r') as f:
        return json.load(f)



In [3]:
annotations = load_annotations(annotations_path)

FileNotFoundError: [Errno 2] No such file or directory: '../mnt/V1/annotations/annotations_preprocessed.json'

In [ ]:
# Calcular distribución de clases
def class_distribution(annotations):
    positions = [annotation['Position'] for annotation in annotations]
    counter = Counter(positions)
    return counter

In [ ]:
# Graficar la distribución
def plot_class_distribution(counter):
    plt.figure(figsize=(12, 6))
    sns.barplot(x=list(counter.keys()), y=list(counter.values()))
    plt.xticks(rotation=45, ha='right')
    plt.title('Distribución de Clases')
    plt.xlabel('Clase')
    plt.ylabel('Cantidad')
    plt.show()

In [ ]:
# Obtener distribución de clases
class_counter = class_distribution(annotations)

# Mostrar distribución
plot_class_distribution(class_counter)


In [ ]:
# Análisis de puntos clave
def analyze_keypoints(annotations):
    confidences = []
    for annotation in annotations:
        for pose in ['Pose1', 'Pose2']:
            confidences.extend([kp[2] for kp in annotation[pose]])
    return np.array(confidences)


In [ ]:
# Graficar distribución de confianzas
def plot_confidence_distribution(confidences):
    plt.figure(figsize=(10, 5))
    sns.histplot(confidences, bins=30, kde=True)
    plt.title('Distribución de Confianza en Puntos Clave')
    plt.xlabel('Confianza')
    plt.ylabel('Frecuencia')
    plt.show()


In [ ]:
# Obtener distribuciones de confianza
confidences = analyze_keypoints(annotations)

# Mostrar la gráfica
plot_confidence_distribution(confidences)


In [ ]:
# Matriz de transiciones entre posiciones
def transition_matrix(annotations):
    transitions = []
    for i in range(len(annotations) - 1):
        from_pos = annotations[i]['Position']
        to_pos = annotations[i + 1]['Position']
        transitions.append((from_pos, to_pos))
    
    transition_df = pd.DataFrame(transitions, columns=['From', 'To'])
    transition_matrix = pd.crosstab(transition_df['From'], transition_df['To'])
    return transition_matrix


In [ ]:
# Graficar matriz de transiciones
def plot_transition_matrix(matrix):
    plt.figure(figsize=(12, 10))
    sns.heatmap(matrix, annot=True, fmt="d", cmap="Blues")
    plt.title('Matriz de Transiciones entre Clases')
    plt.xlabel('Clase de Destino')
    plt.ylabel('Clase de Origen')
    plt.show()

In [ ]:
# Calcular matriz de transiciones
matrix = transition_matrix(annotations)

# Mostrar la gráfica
plot_transition_matrix(matrix)


In [ ]:
# Duración promedio de cada posición
def position_durations(annotations):
    frame_durations = {}
    for annotation in annotations:
        pos = annotation['Position']
        frame = annotation['Frame']
        if pos not in frame_durations:
            frame_durations[pos] = []
        frame_durations[pos].append(frame)
    
    # Calcular duración promedio
    durations = {pos: (max(frames) - min(frames)) for pos, frames in frame_durations.items()}
    return durations

In [ ]:
# Graficar duraciones promedio
def plot_position_durations(durations):
    plt.figure(figsize=(10, 5))
    sns.barplot(x=list(durations.keys()), y=list(durations.values()))
    plt.xticks(rotation=45, ha='right')
    plt.title('Duración Promedio por Posición')
    plt.xlabel('Posición')
    plt.ylabel('Duración (Frames)')
    plt.show()

In [ ]:
# Obtener duraciones promedio
durations = position_durations(annotations)

# Mostrar gráfica
plot_position_durations(durations)
